# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint
from functools import wraps

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter() 
        result = func(*args, **kwargs)
        tm = time.perf_counter() - start
        print(f"Время выполнения функции {func.__name__}: {tm:.17f}")
        return result
        
    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'Функция {func.__name__} вызвана с параметрами:\n {args}, {kwargs}')
        return result

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        # кол-во вызовов храним в объекте декоратора
        wrapper.count += 1
        print(f"Функция была вызывана {wrapper.count} раз")
        return result

    # задаем начальное значение счетчика в декораторе
    wrapper.count = 0
    return wrapper

## Задание 4

In [6]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, 
    чьи аргументы args должны быть хешируемыми
    """
    cache = {}
    @wraps(func)
    def fmemo(*args):
        if args in cache:
            return cache[args]
        else:
            cache[args] = func(*args)
            return cache[args]

    fmemo.cache = cache
    return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 1.77834049000011873
Функция word_count вызвана с параметрами:
 ('whole',), {}
Функция была вызывана 1 раз
Cлово whole встречается 176 раз


In [8]:
# вариант без кэширования
# используется вложенная ф-я, чтобы замерять время выполнения в целом
@benchmark
def fib_benchmark(n):
    def fib(n):
        if n < 2:
            return n
        return fib(n-2) + fib(n-1)
    return fib(n)
    

In [9]:
# измеряем время выполнения
result1 = fib_benchmark(30)

Время выполнения функции fib_benchmark: 0.10972291899815900


In [11]:
# вариант с кэшированием
# используется вложенная ф-я, чтобы замерять время выполнения в целом
@benchmark
def fib_memo(n):
    @memo
    def fib(n):
        if n < 2:
            return n
        return fib(n-2) + fib(n-1)
        
    return fib(n)

In [14]:
# измеряем время выполнения
result2 = fib_memo(30)

Время выполнения функции fib_memo: 0.00003722999827005


Время выполнения рекурсивной реализации расчета чисел Фибоначчи с декоратором memo(func) меньше, чем без него.